In [1]:
from __future__ import print_function

import os.path
import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '14y0FLqGTA6rgvfAmwBeH8bUf_wBKDEHTH8JxXwxtSJI'
SAMPLE_RANGE_NAME = 'FRADEE!A1:Z10000'



In [2]:
def chama_frade():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        values = result.get('values', [])
        values = result.get('values', [])

        
    except HttpError as err:
        print(err)
    
    dados_frade = pd.DataFrame(data=values, columns=values.pop(0))

    return dados_frade

In [ ]:
df = chama_frade()
df.columns

In [ ]:
df.head(2)

In [ ]:
df2 = df[['Ticket','NF','Data Ocorrência','Ocorrência','Parceiro',' Valor ',' Produto ','Emissão','Quantidade','Categoria Produto']]
df2.head(5)

In [ ]:
fradee = df2[df2[' Produto '] != 'Carga Total']
fradee.head(5)

In [ ]:
fradee['Emissão'] = pd.to_datetime(fradee['Emissão'], format='%d/%m/%Y', errors='coerce')
fradee = fradee.sort_values(by='Emissão', ascending=False)
fradee['Emissão'] = fradee['Emissão'].dt.strftime('%d/%m/%Y')

In [ ]:
fradee.head(5)

In [ ]:
fradee.shape

In [ ]:
mestre_bu = pd.read_parquet(r"N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2. Vendas\2.NFs_Venda.parquet")
mestre_bu.columns

In [ ]:
mestre_bu = mestre_bu[['Núm. NF-e', 'Data de Emissão','UF Destino']]
mestre_bu.head(3)

In [ ]:
mestre_bu['Data de Emissão'] = pd.to_datetime(mestre_bu['Data de Emissão'], format='%d/%m/%Y', errors='coerce')
mestre_bu = mestre_bu.sort_values(by='Data de Emissão', ascending=False)
mestre_bu['Data de Emissão'] = mestre_bu['Data de Emissão'].dt.strftime('%d/%m/%Y')

mestre_bu.rename(columns={'Núm. NF-e': 'NF', 'Data de Emissão': 'Emissão'}, inplace=True)
mestre_bu.head(3)

In [ ]:
primeiros_bu = mestre_bu.groupby('NF').first().reset_index()
df_resultado = pd.merge(fradee,primeiros_bu, on=['NF'], how='left')
df_resultado.head(10)

In [ ]:
produtos = pd.read_parquet(r"N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2. Produtos\D_Produtos NeoSolar.parquet")
produtos = produtos[['Nome Produto', 'Codigo SKU']]
produtos.head(3)

In [ ]:
produtos.rename(columns={'Nome Produto': ' Produto ', 'Codigo SKU': 'SKU'}, inplace=True)
produtos.head(5)

In [ ]:
df_resultado.head(20)

In [ ]:
# Encontre o primeiro elemento de cada grupo na tabela "produtos"
primeiros_produtos = produtos.groupby(' Produto ').first().reset_index()
primeiros_produtos.head(5)

In [ ]:
# Faça o merge entre a tabela "fradee" e a versão modificada da tabela "produtos"
Avarias = pd.merge(df_resultado, primeiros_produtos, on=' Produto ', how='left')
Avarias.head(10)

In [ ]:
Avarias.drop(columns='Emissão_y', inplace=True)
Avarias.rename(columns={'Emissão_x': ' Emissão ', 'UF Destino': 'UF'}, inplace=True)
Avarias.head(5)

In [ ]:

# Supondo que você já tenha seu dataframe carregado como Avarias
# Exemplo: Avarias = pd.read_csv('seu_arquivo.csv')

# Convertendo a coluna ' Emissão ' para datetime
Avarias[' Emissão '] = pd.to_datetime(Avarias[' Emissão '], format='%d/%m/%Y').dt.date

# Filtrando para dados posteriores a 01/01/2023
Avarias = Avarias[Avarias[' Emissão '] > pd.to_datetime('2023-01-01').date()]

# Exibindo as primeiras linhas do dataframe filtrado para confirmação
Avarias.head(5)


In [ ]:
Avarias.shape

In [ ]:
Avarias_faltantes = Avarias[Avarias["SKU"].isnull()]
Avarias_faltantes.head(15)

In [23]:
Avarias.to_excel("N:/Drives compartilhados/28 - Dados BI/2 Dados Transformados/2.Frete/2.Frete_Avarias.xlsx")
Avarias_faltantes.to_excel("N:/Drives compartilhados/04 - OPERAÇÕES/70. Transporte/05_Casos de Avarias Extravios e Roubos/Avarias_SKU_faltantes.xlsx")

In [ ]:
Avarias["SKU"].isnull().value_counts()